### Ruben Abbou
## Newton Method for Computing Matrix Inverses
This method requires only additions and multiplication of matrices.

In [1]:
import numpy as np
from numpy.random import randn, rand, randint, seed, uniform
from numpy.linalg import norm, inv, solve, pinv
import matplotlib.pyplot as plt
import math
import time

In [2]:
def newton_inverse(A, e=1e-8, max_steps=1000):
    '''
    inputs:
        - A: initial matrix
    outputs:
        - xk: minimizer of f using newton method
    Runs Newton method for matrix inverse (without steepest descent iterations,
    since we already have a good starting point).
    '''
    alpha = uniform(0, 1/norm(A, ord=2)**2)
    X0 = alpha * A.T # initialize x0 as shown in part (c)
    
    for k in range(max_steps):
        Xk = X0.dot(2 * np.identity(A.shape[0]) - A.dot(X0))
        if norm(Xk - X0) < e:
            return Xk, k
        X0 = np.array(Xk)
    print("Newton algorithm failed to converge") 

In [3]:
def backward_error(A, X):
    try:
        Y = inv(A)
    except:
        Y = pinv(A)
    
    I = np.identity(len(A))
    ex = norm(I-A@X)
    ey = norm(I-A@Y)
    
    return (ex, ey)


## i.
Let's calculate the forward errors for $2\times2$ integer matrices and $10\times10$ diagonal matrices of rational numbers. I used a tolerance of $10^{-8}$.

In [4]:
for _ in range(10):
    A = randint(-5000, 5000, size=(2, 2))
    Xk, k = newton_inverse(A)
    a, b, c, d = A[0, 0], A[0, 1], A[1, 0], A[1, 1]
    detA = (a*d-b*c)
    if detA:
        A_minus_1 = 1/detA*np.array([[d, -b], [-c, a]]) #inverse of a 2x2 matrix
        print(f"Forward error obtained after {k} steps: {norm(Xk - A_minus_1)}")
    else:
        print("A is singular")

Forward error obtained after 4 steps: 7.265469234097228e-19
Forward error obtained after 8 steps: 1.4844286914218873e-15
Forward error obtained after 7 steps: 3.408294302622768e-16
Forward error obtained after 9 steps: 9.373686084755107e-16
Forward error obtained after 5 steps: 1.2428310280723025e-17
Forward error obtained after 9 steps: 1.5739712730627153e-17
Forward error obtained after 14 steps: 1.2361806729070689e-18
Forward error obtained after 22 steps: 1.7355703022298208e-17
Forward error obtained after 9 steps: 2.737476298800061e-19
Forward error obtained after 9 steps: 4.846769225765442e-15


In [5]:
for _ in range(10):
    seed()
    A = np.diag(randint(-5000, 5000, size=10)/randint(-5000, 5000, size=10))
    Xk, k = newton_inverse(A)
    A_minus_1 = np.diag([1/a for a in np.diag(A)]) # inverse of each entry
    print(f"Forward error obtained after {k} steps: {norm(Xk - A_minus_1)}")

Forward error obtained after 19 steps: 2.2898349882893854e-16
Forward error obtained after 13 steps: 1.1102230246251565e-16
Forward error obtained after 14 steps: 6.38529066861312e-16
Forward error obtained after 14 steps: 4.47545209131181e-16
Forward error obtained after 17 steps: 4.577566798522237e-16
Forward error obtained after 15 steps: 1.8343894894033213e-15
Forward error obtained after 14 steps: 3.3335590258932494e-16
Forward error obtained after 12 steps: 5.551115123125783e-17
Forward error obtained after 16 steps: 3.5579140431534355e-15
Forward error obtained after 18 steps: 1.3334236103572998e-15


## ii.

In [6]:
rate = 0
ratet = 0
for n in [10, 100, 1000]:
    print("testing matrix: ", (n, n))
    I = np.identity(n)
    for _ in range(10):
        A = randn(n, n)
        Xk, k = newton_inverse(A)
        tn = time.time()
        be_newton = norm(I-A.dot(Xk))
        tn = time.time() - tn
        print("Backward error obtained after {} steps (duration {:.2e}): {:.2e}"\
              .format(k, tn, be_newton))
        tnp = time.time()
        be_numpy = norm(I-A.dot(inv(A)))
        tnp = time.time() - tnp
        print("Backward error of numpy.linalg.inv (duration {:.2e}): {:.2e}"\
             .format(tnp, be_numpy))
        if tn - tnp < 0:
            print("shorter duration for newton method")
            ratet+=1
        else:
            print("numpy is shorter")
        if be_newton - be_numpy < 0:
            print("better error for newton method!\n")
            rate+=1
        else:
            print("numpy wins\n")

testing matrix:  (10, 10)
Backward error obtained after 19 steps (duration 1.69e-05): 6.45e-15
Backward error of numpy.linalg.inv (duration 1.85e-04): 8.34e-15
shorter duration for newton method
better error for newton method!

Backward error obtained after 14 steps (duration 2.60e-05): 1.62e-15
Backward error of numpy.linalg.inv (duration 1.02e-04): 2.64e-15
shorter duration for newton method
better error for newton method!

Backward error obtained after 17 steps (duration 2.31e-05): 3.51e-15
Backward error of numpy.linalg.inv (duration 7.89e-05): 3.83e-15
shorter duration for newton method
better error for newton method!

Backward error obtained after 17 steps (duration 2.10e-05): 4.77e-15
Backward error of numpy.linalg.inv (duration 7.82e-05): 4.44e-15
shorter duration for newton method
numpy wins

Backward error obtained after 16 steps (duration 2.69e-05): 1.42e-15
Backward error of numpy.linalg.inv (duration 8.30e-05): 1.37e-15
shorter duration for newton method
numpy wins

Backwa

In [7]:
print("Newton wins error {:.2f}% of the time".format(rate/30*100))
print("Newton wins duration {:.2f}% of the time".format(ratet/30*100))

Newton wins error 83.33% of the time
Newton wins duration 100.00% of the time


We can see that out Newton implementation with tolerance of $10^{-8}$ does better than NumPy's inversion function more than $80\%$ of the time. It is therefore incredibly precise, and for all our bigger matrices ($100\times100, 1000\times 1000$), our implementation has a smaller error rates than NumPy.

Additionally, our implementation also takes less time to compute the inverse than NumPy does, for all matrices.